In [25]:
# Import Required Packages
import lxml
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import urllib
import urllib.request as urllib2
import os
import requests
import pandas as pd

In [26]:
# Get URL
url = 'https://virginiasports.com/roster.aspx?roster=1142&path=softball'
# establish a session to request website
user_agent = {'User-agent': 'Mozilla/5.0'}
urlSplit = url.split('/')
# Create string to represent the base url
baseUrl = urlSplit[0] + '/' + urlSplit[1] + '/' + urlSplit[2]

In [27]:
# Check if we get a response
r = requests.get(url, headers = user_agent)
# Make the soup
soup = BeautifulSoup(r.text, 'html.parser')

In [28]:
# Create a list of the players on the roster.
players = list()
firstName = list()
lastName = list()
numbers = list()
bat = list()
throw = list()
pos = list()

# Scrape the players name
for item in soup.findAll('td', {"class": "sidearm-table-player-name"}):
    name = item.text
    players.append(item.text)
    if('  ' in name):
        fullName = name.split('  ')
    else: fullName = name.split(' ')
    firstName.append(fullName[0])
    lastName.append(fullName[1])
players

# Scrape te players jersey number
for item in soup.findAll('td', {"class": "roster_jerseynum"}):
    numbers.append(item.text)
    
    
# Scrape the players batting/throwing side    
for item in soup.findAll('td', {"class": 'sidearm-table-custom-field'}):
    b_t = item.text.split('/')
    if(b_t[0] == 'R'):
        bat.append('Right')
    else: bat.append('Left')
    if(len(b_t) == 2):
        if(b_t[1] == 'R'):
            throw.append('Right')
        else: throw.append('Left')
    else: throw.append('Right')
        
# Scrape the players position
for item in soup.findAll('td', {"class": "rp_position_short"}):
    pos.append(item.text)
    
    
#########################################################
# Get user input for Team Abbrieviation:
#########################################################
ta = 'UVA'

#########################################################
# Get user input for Team Organization:
#########################################################
to = 'University of Virginia'

In [29]:
# Construct the Data Frame and Display
frame = pd.DataFrame(numbers, columns=['Jersey'])
frame['First Name'] = firstName
frame['Last Name'] = lastName
frame['Pitching Strength'] = throw
frame['Hitting Strength'] = bat
frame['Position'] = pos
frame['Team Abbrieviation'] = ta
frame['Team Organization'] = to
frame

,Jersey,First Name,Last Name,Pitching Strength,Hitting Strength,Position,Team Abbrieviation,Team Organization
0,1,Emma,McBride,Right,Right,IF,UVA,University of Virginia
1,2,Savana,Avilla,Right,Left,IF/OF,UVA,University of Virginia
2,3,Bailey,Winscott,Right,Left,OF,UVA,University of Virginia
3,4,Tori,Gilbert,Right,Right,OF,UVA,University of Virginia
4,5,Mikaila,Fox,Right,Right,IF,UVA,University of Virginia
5,6,Madi,Wilson,Right,Right,C/IF,UVA,University of Virginia
6,7,Rachel,Keshin,Right,Right,OF,UVA,University of Virginia
7,8,Morgan,Murphy,Right,Right,P,UVA,University of Virginia
8,9,Ashley,Jennings,Right,Right,C,UVA,University of Virginia
9,10,Kate,Covington,Right,Right,OF,UVA,University of Virginia


In [30]:
# create a list of links to player pages and display
playerAnchor = list()
anchors = soup.select('td.sidearm-table-player-name a', href = True)
for a in anchors:
    playerAnchor.append(baseUrl + a['href'])
playerAnchor

['https://virginiasports.com/roster.aspx?rp_id=6673',
 'https://virginiasports.com/roster.aspx?rp_id=6664',
 'https://virginiasports.com/roster.aspx?rp_id=6678',
 'https://virginiasports.com/roster.aspx?rp_id=6668',
 'https://virginiasports.com/roster.aspx?rp_id=7176',
 'https://virginiasports.com/roster.aspx?rp_id=6677',
 'https://virginiasports.com/roster.aspx?rp_id=6672',
 'https://virginiasports.com/roster.aspx?rp_id=7179',
 'https://virginiasports.com/roster.aspx?rp_id=6670',
 'https://virginiasports.com/roster.aspx?rp_id=6666',
 'https://virginiasports.com/roster.aspx?rp_id=6676',
 'https://virginiasports.com/roster.aspx?rp_id=7182',
 'https://virginiasports.com/roster.aspx?rp_id=7181',
 'https://virginiasports.com/roster.aspx?rp_id=6665',
 'https://virginiasports.com/roster.aspx?rp_id=6675',
 'https://virginiasports.com/roster.aspx?rp_id=6680',
 'https://virginiasports.com/roster.aspx?rp_id=6674',
 'https://virginiasports.com/roster.aspx?rp_id=7177',
 'https://virginiasports.com

In [31]:
#########################################################
# User defines the path of where they save the file in Path
#########################################################

Path = './uva/'

# find players and download images.
for idx, playerLink in enumerate(playerAnchor):
    # reference players link and create soup
    r = requests.get(playerLink, headers = user_agent)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # find images of the players
    player = soup.find('img', {'alt': players[idx]})
    print(player)
    # download the image
    if(player != None):
        imgurl = baseUrl + player['src'].split('?')[0]
        urllib2.urlretrieve(imgurl, os.path.join(Path, os.path.basename(imgurl)))

<img alt="Emma McBride" src="/images/2018/11/9/Emma_McBride.jpg?width=300"/>
<img alt="Savana Avilla" src="/images/2019/10/8/Avilla_Savana_19M.JPG?width=300"/>
<img alt="Bailey Winscott" src="/images/2018/11/9/Bailey_Winscott.jpg?width=300"/>
<img alt="Tori Gilbert" src="/images/2018/11/9/Tori_Gilbert.jpg?width=300"/>
<img alt="Mikaila Fox" src="/images/2019/10/8/Fox_Mikaila_19M.JPG?width=300"/>
<img alt="Madi Wilson" src="/images/2019/10/8/Wilson_Madi_19M.JPG?width=300"/>
<img alt="Rachel Keshin" src="/images/2019/10/8/Kesin_Rachel_19M.JPG?width=300"/>
<img alt="Morgan Murphy" src="/images/2019/10/8/Murphy_Morgan_19M.JPG?width=300"/>
<img alt="Ashley Jennings" src="/images/2018/5/14/12867228.jpeg?width=300"/>
<img alt="Kate Covington" src="/images/2019/10/8/Covington_Kate_19M.JPG?width=300"/>
<img alt="Madi Traver" src="/images/2019/10/8/Traver_Madi_19M.JPG?width=300"/>
<img alt="Riley Wilkinson" src="/images/2019/10/8/Wilkinson_Riley_19M.JPG?width=300"/>
<img alt="Katie Haney" src="/

In [32]:
frame.to_csv(os.path.join(Path, 'roster.csv'))